# Linear Regression
This is my first Project on Linear Regression. In this Project I will try to predict the car fuel effeciency based on current data using my linear regression model.

There are several steps involved in it I have a decent amount of data which I got from https://www.fueleconomy.gov/feg/download.shtml which contains information related to cars and there mpg(miles per gallon).

The problem with this data is although it is excel sheet and csv file. There is still a small amount of incononsistencies in the data. 

Our first few steps will involve making our data uniform and properly labelled, so that our model can easily and effectively train on the model.

Right now I have data from 2000 till 2025. The first problem involves, how can I make the data in a uniform file format, because our data is in mixed files.

Since the majority of data files are in excel format. I will go ahead and make the other .csv files to .xlsx files.

In [1]:
import os
import pandas as pd

In [2]:
# in the bellow cell, we will convert all the .csv files to .xlsx files in order to achieve uniform file format for our data
csv_directory = 'Data/'

for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        
        csv_file = os.path.join(csv_directory, filename)
        df = pd.read_csv(csv_file, on_bad_lines='skip')

        
        
        excel_file = os.path.join(csv_directory, filename.replace('.csv', '.xlsx'))

        df.to_excel(excel_file, index=False)
        print(f'Converted {filename} to {excel_file}')

        os.remove(csv_file)
        print(f'Deleted {csv_file}')

Converted 2006_FE_Guide_14-Nov-2005_download.csv to Data/2006_FE_Guide_14-Nov-2005_download.xlsx
Deleted Data/2006_FE_Guide_14-Nov-2005_download.csv
Converted 2008_FE_guide_ALL_rel_dates_-no sales-for DOE-5-1-08.csv to Data/2008_FE_guide_ALL_rel_dates_-no sales-for DOE-5-1-08.xlsx
Deleted Data/2008_FE_guide_ALL_rel_dates_-no sales-for DOE-5-1-08.csv
Converted guide_2003_feb04-03b.csv to Data/guide_2003_feb04-03b.xlsx
Deleted Data/guide_2003_feb04-03b.csv
Converted guide2005-2004oct15.csv to Data/guide2005-2004oct15.xlsx
Deleted Data/guide2005-2004oct15.csv
Converted gd04-Feb1804-RelDtFeb20.csv to Data/gd04-Feb1804-RelDtFeb20.xlsx
Deleted Data/gd04-Feb1804-RelDtFeb20.csv


It is visible from the above output that all the files has been converted, now we will proceed with our next inconsistency which is our file names. As you can see all the files have long and different names, which might become a peoblem when we will merge them into one (This is only an assumption). I think the better way to change the names of our file will be if we name it with the year's data it consist. I will try my best to keep this automated but looking at some files. It is not possible, so after automated task, I will perform the remaining manually.

If we notice the patteren most of the recent files follow the same pattren in their names, which will be very helpful to automate the task. The name provide too much information like; release date, etc. This information is not of any use for our application. The only thing we need is year. 

In [6]:
# in the below method we will change the file names for our convinience to work with simplified data

for file in os.listdir("Data/"):
    original = file
    file, extention = file.split(".")
    file = file.replace("_","-")
    file = file.split("-")
    file = file[0].split()
    file = file[0]
    os.rename("Data/"+original, "Data/"+file+"."+extention)
    
print(os.listdir("Data/"))

['2014.xlsx', '2016.xlsx', '2022.xlsx', '2020.xlsx', '2019.xlsx', '2008.xlsx', '2023.xlsx', '2021.xlsx', '2001.xls', '2017.xlsx', '2011.xlsx', '2002.xls', '2015.xlsx', '2003.xlsx', '2013.xlsx', '2004.xlsx', '2018.xlsx', '2012.xlsx', '2024.xlsx', '2006.xlsx', '2025.xlsx', '2007.xls', '2005.xlsx', '2010.xlsx', '2009.xls']
